In [45]:
!mkdir cropped_lps
!mkdir _oof

import os


ccpd_base_path = 'CCPD_data/ccpd_base/'
cropped_path = 'cropped_lps/'

uncropped_images = os.listdir(ccpd_base_path)
uncropped_images = [ccpd_base_path + img for img in uncropped_images]

A subdirectory or file cropped_lps already exists.


A subdirectory or file _oof already exists.


In [46]:
provinces = ["皖", "沪", "津", "渝", "冀", "晋", "蒙", "辽", "吉", "黑", "苏", "浙", "京", "闽", "赣", "鲁", "豫", "鄂", "湘", "粤", "桂", "琼", "川", "贵", "云", "藏", "陕", "甘", "青", "宁", "新", "警", "学", "O"]
alphabets = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W',
             'X', 'Y', 'Z', 'O']
ads = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'J', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X',
       'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'O']

def extract_plate_number(plate_number):
    chi_let = provinces[int(plate_number.split("_")[0])]
    alp_let = alphabets[int(plate_number.split("_")[1])]
    alp_num_let = plate_number.split("_")[2:]
    alp_num_let = "".join([ads[int(char)] for char in alp_num_let])
    all_let = chi_let + alp_let + alp_num_let
    return all_let

In [47]:
from ultralytics import YOLO
import cv2

yolo_model = YOLO('runs/detect/train/weights/best.pt')

def get_crop(image, box):
    x1, y1, x2, y2 = box
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
    return image[y1:y2, x1:x2]

def detect_and_crop(image_path):
    # Original Image
    ori_image = cv2.imread(image_path)
    ori_image = cv2.cvtColor(ori_image, cv2.COLOR_BGR2RGB)

    # output name
    plate_number = image_path.split("-")[4]

    result = yolo_model.predict([image_path], verbose=False)[0]
    boxes = [box.data[0][:4] for box in result.boxes]
    num_boxes = len(boxes)

    if num_boxes == 1:
        crop = get_crop(ori_image, boxes[0])
        cv2.imwrite(cropped_path + plate_number + ".jpg", cv2.cvtColor(crop, cv2.COLOR_RGB2BGR))

for image in uncropped_images:
    detect_and_crop(image)

KeyboardInterrupt: 